!pip install "pyautogen[gemini]~=0.2.0b4"

In [1]:
import requests
import json
import pdb
import os
import re

from typing import Any, Callable, Dict, List, Optional, Tuple, Type, Union

import autogen
from autogen import AssistantAgent, Agent, UserProxyAgent, ConversableAgent

from autogen.agentchat.contrib.img_utils import get_image_data, _to_pil
from autogen.agentchat.contrib.multimodal_conversable_agent import MultimodalConversableAgent

from termcolor import colored
import random

from autogen.code_utils import DEFAULT_MODEL, UNKNOWN, content_str, execute_code, extract_code, infer_lang

/home/user/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gemini-pro"],
    },
)

In [3]:
llm_config = {"config_list": config_list, "cache_seed": 42}
user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    code_execution_config={"last_n_messages": 3, "work_dir": "groupchat"},
    human_input_mode="NEVER",
)
coder = autogen.AssistantAgent(
    name="Coder",
    system_message="""You are an expert in extracting product quantity details from titles. Given a product title, identify relevant details and output them in the following JSON format with appropriate field labels:
                    {'unit_of_measure': , 'count_per_pack': , 'multi_quantity':, 'total_quantity':}
                    Guidelines:
                    If count per pack is not given in the provided value and multi quantity is given, set count per pack to 1; if multi quantity is not given and count per pack is given, set multi quantity to 1.
                    """,
    llm_config=llm_config,
    max_consecutive_auto_reply=1
)
pm = autogen.AssistantAgent(
    name="Product_manager",
    system_message="""Calculate total quantity is equal to count per pack * multi quantity""",
    llm_config=llm_config,
    max_consecutive_auto_reply=1
)
groupchat = autogen.GroupChat(agents=[user_proxy, coder, pm], messages=[], max_round=10,allow_repeat_speaker=False)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [8]:
task = "V8 +Energy Lightly Carbonated, Blackberry Cranberry, 12 Ounce, 4 Count"

In [9]:
results = user_proxy.initiate_chat(
    manager, message=task)
# type exit to terminate the chat

User_proxy (to chat_manager):

V8 +Energy Lightly Carbonated, Blackberry Cranberry, 12 Ounce, 4 Count

--------------------------------------------------------------------------------


Model gemini-pro not found. Using cl100k_base encoding.


Model gemini-pro not found. Using cl100k_base encoding.


Coder (to chat_manager):

```
{
  "unit_of_measure": "Ounce",
  "count_per_pack": 4,
  "multi_quantity": 1,
  "total_quantity": 48
}
```

The total quantity is calculated by multiplying the count per pack by the multi-quantity.

Count per pack = 4
Multi quantity = 1

Total quantity = Count per pack * Multi quantity
Total quantity = 4 * 1
Total quantity = 4

Since the unit of measure is "Ounce", the total quantity is 48 Ounces.

--------------------------------------------------------------------------------


Model gemini-pro not found. Using cl100k_base encoding.


In [6]:
# user_proxy.send(
#     recipient=manager,
#     message="""""",
# )

In [7]:
print(results)

None
